In [ ]:
!pip install openai
!pip install colorama
!pip install tqdm

In [45]:
# Import Necessary SDKs
import os
from openai import OpenAI
import json
import pandas as pd
from colorama import Fore, Style
import time
from tqdm import tqdm

In [46]:
token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx_xxxxxxxxxxxxxxxxxxxxxx"
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"
csv_path = "grounding_truth.csv"

In [47]:
client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

System_Prompt = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n
Observe attentivement la facture et extrait les informations suivantes:  \n

- Fournisseur : C'est le nom du fournisseur qui a vendu des produits ou des services à un client. Généralement, C'est dans le Logo du Fournisseur ou a côté. \n
- Date_Facture : C'est la date de la facture. \n
- Num_Facture : C'est le numéro de la facture se forme d'une série de numéros ou numéros avec lettres. \n
- TTC : C'est le montant total que le client va payer, peut etre referer avec : Total TTC, Net à Payer ou quelque chose similaire. \n
- TT  : C'est le montant total de la facture hors taxes, peut etre referer avec : Total HT ou quelque chose similaire. \n
- TVA : C'est le montant de TVA appliqué au Total hors taxes, peut etre referer avec : TVA, Total TVA, Taxe ou quelque chose similaire. \n

- Votre reponse est un String se la forme d'un dictionaire python avec les clés-valeurs suivantes : \n

     "Invoice_Name": le nom et format du l'image du la facture,
     "Fournisseur": "Fournisseur",
     "Date_Facture": "Date_Facture",
     "Num_Facture": "Num_Facture",
     "TTC": "TTC",
     "TT": "TT",
     "TVA": "TVA"

- Ne pas formater votre reponse comme un code python , repondre juste en texte. \n
- Les 3 prix TT,TTC,TVA doivent être des chiffres sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""

[Dataset](https://huggingface.co/datasets/Noureddinesa/Images_Test)


In [48]:
def Save_to_csv(json_data):
    data = json.loads(json_data)
    df = pd.DataFrame([data])

    if os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(csv_path, index=False)

    print(Fore.GREEN + Style.BRIGHT + "CSV Updated Successfully" + Style.RESET_ALL)


def Run_ALL(Names):
  counter = 0
  for Name in tqdm(Names,desc="Processing Invoices", unit="number"):
        Hugg_path = "https://huggingface.co/datasets/Noureddinesa/Images_Test/resolve/main/"+Name
        response = client.chat.completions.create(
                      model=model_name,
                      messages=[
                          {"role": "system", "content": System_Prompt},
                          {
                          "role": "user",
                          "content": [
                            {"type": "text", "text": f"Extraire les Informations demandées a partir de la facture {Name} suivante : "},
                            {
                              "type": "image_url",
                              "image_url": {"url":Hugg_path},
                            },
                          ],
                          }
                            ],
                      #max_tokens=500,
                    )

        data = response.choices[0].message.content
        print(data)
        Save_to_csv(data)
        # Sleep time between requests of 2 seconds
        time.sleep(2)
        counter += 1
        # We set a Sleep of 1 min so we don't exceed the rate limit (10 requests/min)
        if counter == 7:
          time.sleep(60)
          counter = 0

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

project_path = "/content/drive/MyDrive/Textra_Insights/"
Folder_path = project_path+"Test_Invoices"
Files = os.listdir(Folder_path)
sorted_files = sorted(Files, key=lambda x: int(x.split('.')[0]))
sorted_files

In [51]:
Run_ALL(sorted_files)

Processing Invoices:   0%|          | 0/30 [00:00<?, ?number/s]

{
    "Invoice_Name": "2.jpg",
    "Fournisseur": "OUTIDIS SARL",
    "Date_Facture": "13/01/2023",
    "Num_Facture": "FV230007",
    "TTC": 586.80,
    "TT": 489.00,
    "TVA": 97.80
}
CSV Updated Successfully


Processing Invoices:   3%|▎         | 1/30 [00:05<02:31,  5.24s/number]

{
    "Invoice_Name": "3.jpg",
    "Fournisseur": "FUN PLACE",
    "Date_Facture": "14/04/2023",
    "Num_Facture": "FUNPLACE - 2023/000403",
    "TTC": 3864,
    "TT": 3220,
    "TVA": 644
}
CSV Updated Successfully


Processing Invoices:   7%|▋         | 2/30 [00:10<02:23,  5.14s/number]

...

Processing Invoices:  93%|█████████▎| 28/30 [06:25<00:49, 24.78s/number]

{
    "Invoice_Name": "417.jpg",
    "Fournisseur": "KAYAR",
    "Date_Facture": "11/05/2023",
    "Num_Facture": "F2023/00297",
    "TTC": 13777.31,
    "TT": 11481.09,
    "TVA": 2296.22
}
CSV Updated Successfully


Processing Invoices:  97%|█████████▋| 29/30 [06:30<00:18, 18.87s/number]

{
    "Invoice_Name": "419.jpg",
    "Fournisseur": "PRO NUMERIQUE",
    "Date_Facture": "01/02/2023",
    "Num_Facture": "FA2023043",
    "TTC": 1300,
    "TT": 1083.33,
    "TVA": 216.67
}
CSV Updated Successfully


Processing Invoices: 100%|██████████| 30/30 [06:35<00:00, 13.19s/number]
